## Selecteren van gegevens uit PostgreSQL database tabellen met SQL

In de vorige paragrafen hebben we gezien hoe je data uit verschillende bestandsformaten kan laden in een database tabel. Als de data in een database tabel staat, kunnen we deze data met [SQL queries](https://nl.wikipedia.org/wiki/Select_(SQL) opvragen uit de database. Via SQL queries selecteer je nul, één of meer rijen uit één of meerdere tabellen op basis van nul, één of meer condities. SQL queries kunnen uitgevoerd worden via _psycopg2_ (Python interface). De queries kunnen ook uitgevoerd worden met tools zoals [pgAdmin](https://www.pgadmin.org/) of [DBeaver](https://dbeaver.io/). Om een SQL query uit te kunnen voeren, moet je eerst een connectie met de database maken. Vervolgens kan je de SQL query uitvoeren.

Hieronder volgen een aantal voorbeelden van SQL select queries. 

Selecteren van alle rijen uit een tabel _meteostation_. Per rij worden alle kolommen getoond.

```{sql, eval=F}
SELECT * FROM meteostation
```
<br/>
Selecteren van alle rijen uit een tabel _meteostation_. Per rij wordt een beperkte set van kolommen getoond.

```{sql, eval=F}
SELECT id, hoogte, naam, eigenaar FROM meteostation
```
<br/>
Tellen van aantal rijen in tabel _meteostation_. 

Hiervoor gebruiken we de functie [COUNT](https://www.w3schools.com/sql/sql_count_avg_sum.asp).

```{sql, eval=F}
SELECT COUNT(1) FROM meteostation
```
<br/>
Selecteren van rij voor meteostation \'Schiphol\' uit tabel _meteostation_. 

In de WHERE-clause geven we aan dat we alleen geïnteresseerd zijn in het meteostation Schiphol. 

```{sql, eval=F}
SELECT * FROM meteostation WHERE naam = 'Schiphol'
```

Als we meerdere stations willen selecteren kunnen we gebruik maken van [OR](https://www.w3schools.com/sql/sql_and_or.asp) of van [IN](https://www.w3schools.com/sql/sql_in.asp).

```{sql, eval=F}
SELECT * FROM meteostation WHERE naam = 'Schiphol' OR naam = 'Valkenburg
```

```{sql, eval=F}
SELECT * FROM meteostation WHERE naam IN ('Schiphol', 'Valkenburg')
```
<br/>
Selecteren van gemiddelde_temperatuur waarnemingen tussen 01-01-2016 en 31-01-2016 uit de tabel _waarneming_. 

Om binnen een datuminterval te zoeken, moeten we een minimale en maximale datum opgeven en deze condities samenvoegen met [AND](https://www.w3schools.com/sql/sql_and_or.asp) omdat de datum aan beide condities moet voldoen. Om te zorgen dat de datums waarbinnen we willen zoeken herkend worden als datum, gebruiken we de functie [TO_DATE](https://www.postgresqltutorial.com/postgresql-to_date/). 

```{sql, eval=F}
SELECT * FROM waarneming WHERE datum >= TO_DATE('20160101','YYYYMMDD') AND datum <= TO_DATE('20160131','YYYYMMDD')
```
<br/>

Selecteren van minimale, maximale en gemiddelde hoogte van alle meteostations in tabel _meteostation_. 
Hiervoor gebruiken we de aggregatie functies [MIN](https://www.w3schools.com/sql/sql_min_max.asp), [MAX](https://www.w3schools.com/sql/sql_min_max.asp) en [AVG](https://www.w3schools.com/sql/sql_count_avg_sum.asp). Deze worden uitgevoerd op alle geselecteerde kolomwaardes.

```{sql, eval=F}
SELECT MIN(hoogte), MIN(hoogte), AVG(hoogte) FROM meteostation 
```
<br/>
Koppelen van rijen uit de tabellen _meteostation_ en _waarneming_ op basis van het ID van het meteostation. 

Hierbij wordt de foreign key kolom _meteostation_id_ gekoppeld van de tabel _waarneming_ gekoppeld aan de primary key kolom _id_ van de tabel _meteostation_. Dit gebeurt in de WHERE-clause van de query. Dit wordt een table join genoemd. Op deze manier kan gegevens uit beide tabellen gekoppeld worden. In dit geval willen we per waarneming ook de naam van het meteostation ophalen en tonen. Omdat we kolommen selecteren uit verschillende tabellen en kolommen in verschillende tabellen dezelfde naam kunnen hebben, is het gebruikelijk om in het geval van table joins de naam van de tabel te plaatsen voor de kolomnaam gevolgd door een punt(.), dus \<tabelnaam\>.\<kolomnaam\>. Je doet dit zowel in de SELECT als in de WHERE-clause. Als tabelnamen erg lang worden, kan je ook werken met een [tabel alias](https://www.tutorialspoint.com/postgresql/postgresql_alias_syntax.htm).

```{sql, eval=F}
SELECT meteostation.naam
,      waarneming.gemiddelde_temperatuur
FROM meteostation
,    waarneming
WHERE meteostation.id = waarneming.meteostation_id
```

Een andere manier om tabellen te koppelen in door gebruik te maken van het keyword [INNER JOIN](https://www.w3schools.com/sql/sql_join_inner.asp) of [LEFT JOIN](https://www.w3schools.com/sql/sql_join_left.asp). Bij een INNER JOIN worden alleen de meteostations getoond waarvoor er gemiddele temperaturen zijn gevonden in de tabel _waarneming_. Bij een LEFT JOIN worden alle meteostations getoond. Als er geen gemiddelde temperatuur gevonden is in de tabel _waarneming_ blijft deze kolom leeg (NULL).

```{sql, eval=F}
SELECT meteostation.naam
,      waarneming.gemiddelde_temperatuur
FROM meteostation
INNER JOIN waarneming
ON meteostation.id = waarneming.meteostation_id
```

```{sql, eval=F}
SELECT meteostation.naam
,      waarneming.gemiddelde_temperatuur
FROM meteostation
LEFT JOIN waarneming
ON meteostation.id = waarneming.meteostation_id
```
<br/>
Berekenen van de maximum temperatuur per meteostation uit de tabel _meteostation_ op basis van de gegevens in de tabel _waarneming_ voor de periode waarvoor temperatuurgegevens beschikbaar zijn. De resultaten moeten gesorteerd worden op basis van maximum temperatuur waarbij het station met de hoogste maximum temperatuur als eerste getoond wordt.

Hiervoor moeten we eerst de twee tabellen koppelen zoals in het vorige voorbeeld. Vervolgens moeten we per meteostation de temperaturen gaan groeperen. Hiervoor maken we gebruik van [GROUP BY](https://www.tutorialspoint.com/postgresql/postgresql_group_by.htm). Vervolgens gebruiken we de functie [MAX](https://www.w3schools.com/sql/sql_min_max.asp) op per groep (meteostation) de maximum waarde van alle waardes in de kolom _gemiddelde\_temperatuur_ te berekenen. Als laatste stap moeten we het resultaat sorteren op maximum temperatuur. Hiervoor maken we gebruik van [ORDER BY](https://www.tutorialspoint.com/postgresql/postgresql_order_by.htm). Met het keyword DESC geven we aan dat we descending willen sorteren, dus eerst de hoogste waarde.

```{sql, eval=F}
SELECT meteostation.naam
,      MAX(waarneming.gemiddelde_temperatuur)
FROM meteostation
,    waarneming
WHERE meteostation.id = waarneming.meteostation_id
GROUP BY meteostation.naam
ORDER BY MAX(waarneming.gemiddelde_temperatuur) DESC
```

**Let op:**
Alle kolommen die in de SELECT gebruikt worden, moeten in de GROUP BY gebruikt worden met uitzondering van de kolommen die in een aggregatie functie (bijv. MIN, MAX, AVG, SUM) gebruikt worden.
<br/>
Selecteren van alle meteostations binnen een buffer van 25 km rondom het punt met coördinaat x = 140570 en y = 516299. De coördinaat is in RD ([EPSG:28992](https://epsg.io/28992)). 

Hiervoor moeten we als eerste stap de opgegeven coördinaat omzetten naar een PostGIS punt in het juiste coördinaatsysteem. Hiervoor gebruiken we de functie [ST_MakePoint](https://postgis.net/docs/ST_MakePoint.html) en [ST_setSRID](https://postgis.net/docs/ST_SetSRID.html). Vervolgens berekenen we de buffer om dit punt met de functie [ST_Buffer](https://postgis.net/docs/ST_Buffer.html). Hierbij geven we de grootte van de buffer op in meters (25000). Als laatste stap moeten we de intersectie bepalen tussen de locaties van de meteostations en de buffer, dus welke meteostations vallen binnen de buffer? Hiervoor gebruiken we de functie [ST_Intersects](https://postgis.net/docs/ST_Intersects.html). Deze functie geeft TRUE terug als er een punt binnen de buffer valt en FALSE als een punt buiten de buffer valt. Omdat de functie ST_Intersects vereist dat beide geometrieën in hetzelfde coördinaatsysteem gedefinieerd zijn, moeten we een coördinaattransformatie uitvoeren zodat de locatie van de meteostations, opgeslagen is in [WGS84](https://epsg.io/4326) omgezet wordt naar RD (28992). Hiervoor gebruiken we de functie [ST_Transform](https://postgis.net/docs/ST_Transform.html).

```{sql, eval=F}
SELECT id, naam, hoogte, geom 
FROM meteostation
WHERE ST_Intersects(ST_Transform(geom,28992),ST_Buffer(ST_SetSRID(ST_MakePoint(140570,516299),28992),25000)) 
```
<br/>
Selecteren van meteostations voor de provincie Utrecht op basis van locatie waarbij resultaat bestaat uit lijst met kolom _provincie_ en _meteostation_. 

Hiervoor gebruiken we wederom de functie [ST_Intersects](https://postgis.net/docs/ST_Intersects.html) waarbij we nu alleen de meteostations selecteren die binnen de pronvincie Utrecht vallen. In de WHERE-clause geven we aan dat we alleen geïnteresseerd zijn in de provincie Utrecht. 

```{sql, eval=F}
SELECT meteostation.id, meteostation.naam, meteostation.hoogte, meteostation.geom 
FROM meteostation,
,    province
WHERE ST_Intersects(provincie.geom, meteostation.geom)
AND   provincie.naam = 'Utrecht'
```